Silent Night is a modular malware. This notebook is going to focus on the loader and some of the main component. It also uses code obfuscation, so I am going to skip a lot of the disassembly listings.

## Initial Setup

1. Download the Silent Night version 2.1.7.0 loader reference sample from [MalwareBazaar](https://bazaar.abuse.ch/sample/fbd60fffb5d161e051daa3e7d65c0ad5f589687e92e43329c5c4c950f58fbb75/).

In [1]:
# Install required Python modules
import sys
!{sys.executable} -m pip install --break-system-packages pefile
!{sys.executable} -m pip install --break-system-packages pycryptodome

# Import required Python modules
import datetime
import pprint
import re
import struct
import time

import pefile

from Crypto.Cipher import PKCS1_v1_5
from Crypto.PublicKey import RSA
from Crypto.Cipher import ARC4

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Load the unpacked loader sample
fp = open("fbd60fffb5d161e051daa3e7d65c0ad5f589687e92e43329c5c4c950f58fbb75.bin", "rb")
loader = fp.read()
fp.close()

pe_loader = pefile.PE(data=loader)

## Resolve Windows API Calls

Windows API calls are resolved using an XOR key based hashing algorithm.

Pseudocode:

      ... 
      for ( j = 0; (a1_lt_a2_667(j, name_len) & 1) != 0; ++j )
      {   
        v3 = j;
        LOBYTE(v3) = *(v36 + j); 
        name[j] = to_upper(v3);
      }   
      name[name_len] = 0;
      test_hash_val = api_hashing_alg(name, name_len, 0x2FA9CD05u);
      ... 

Disassembly:

    ...
    .text:00000000000F7911 44 8B 05 CC 31 01 00          mov     r8d, cs:g_api_hash_xor_key
    .text:00000000000F7918 48 8B 94 24 00 01 00 00       mov     rdx, [rsp+1C8h+name_len]
    .text:00000000000F7920 48 8D 8C 24 10 01 00 00       lea     rcx, [rsp+1C8h+name]
    .text:00000000000F7928 E8 53 40 00 00                call    api_hashing_alg
    ...

Pseudocode:

    __int64 __fastcall api_hashing_alg(unsigned __int8 *name, unsigned __int64 name_len, unsigned int xor_key)
    {
      int i; // [rsp+28h] [rbp-20h]
      int hash_val; // [rsp+2Ch] [rbp-1Ch]

      if ( name_len == -1LL )
        name_len = Str::_LengthA_0(name);                                                    
      hash_val = 0;
      if ( name && name_len )
      {
        for ( i = 0; i < name_len; ++i )
        {
          hash_val = *name + 1 + 16 * hash_val;                                                 
          if ( (~hash_val | 0xFFFFFFF) != 0xFFFFFFFF )
            hash_val = xor_0x19182A30(0x16E7D5CF) & ((~(~hash_val | 0xFFFFFFFu) >> 24) ^ hash_val);// 0xfffffff
          ++name;
        }
      }
      return hash_val ^ xor_key;
    }

In [3]:
def resolve_func_by_hash(name, xor_key):
    hash_val = 0

    for c in name.upper():
        hash_val = (c + 1 + 16 * hash_val) & 0xffffffff
        
        math1 = (~hash_val) & 0xffffffff
        math2 = (math1 | 0xfffffff) & 0xffffffff
        if math2 != 0xffffffff:
            math3 = (~math2) & 0xffffffff
            math4 = (math3 >> 24) & 0xffffffff
            math5 = (math4 ^ hash_val) & 0xffffffff
            hash_val = (math5 & 0xfffffff) & 0xffffffff

    hash_val = (hash_val ^ xor_key)
    return hash_val


# extract XOR key
match = re.search(rb'\x44\x8b\x05(?P<offset>[\s\S]{4})', pe_loader.get_memory_mapped_image())
rip = match.start() + pe_loader.OPTIONAL_HEADER.ImageBase
offset = struct.unpack("I", match.group("offset"))[0]
instr_size = 7
xor_key_addr = rip + offset + instr_size
xor_key_rva = xor_key_addr - pe_loader.OPTIONAL_HEADER.ImageBase
xor_key = pe_loader.get_dword_at_rva(xor_key_rva)
print(f"xor_key: 0x{xor_key:x}")

for example in [b"ExitProcess", b"GetProcessHeap", b"GetModuleFileNameW", b"CreateProcessW", b"HttpOpenRequestA"]:
    func_hash = resolve_func_by_hash(example, xor_key)
    print(f"{example}: 0x{func_hash:08x}")

xor_key: 0x2fa9cd05
b'ExitProcess': 0x29669931
b'GetProcessHeap': 0x2f979734
b'GetModuleFileNameW': 0x2040b70d
b'CreateProcessW': 0x2046eb6d
b'HttpOpenRequestA': 0x22822a47


## Decrypt Strings

Strings are encrypted using a hardcoded XOR key.

Pseudocode:

    __int64 __fastcall CryptedStrings::_getA(__int64 encbuf, __int64 plainbuf)
    {   
      char v3; // [rsp+28h] [rbp-20h]
      int i; // [rsp+2Ch] [rbp-1Ch]

      if ( !plainbuf )
        return encbuf;
      for ( i = 0; ; i += xor_0x19182A30(0x19182A31) )// 1
      {   
        v3 = *(encbuf + i); 
        *(plainbuf + i) = g_enc_str_key[i % xor_0x19182A30(0x19182A21)] ^ v3;// 17
        if ( (a1_equals_a2_843(*(plainbuf + i), 0) & 1) != 0 ) 
          break;
      }   
      return plainbuf;
    }   

Disassembly:

    ... 
    .text:00000000000EF651 48 8D 0D A8 B3 01 00          lea     rcx, g_enc_str_key
    .text:00000000000EF658 0F BE 0C 11                   movsx   ecx, byte ptr [rcx+rdx]
    ... 

In [4]:
def decrypt_str(xor_key, encbuf, wide=False):
    plainbuf = b""
    
    for i, eb in enumerate(encbuf):
        pb = eb ^ xor_key[i % len(xor_key)]
        plainbuf += struct.pack("B", pb)
        
    if wide:
        plainbuf = plainbuf.split(b"\x00\x00")[0].replace(b"\x00", b"").decode()
    else:
        plainbuf = plainbuf.split(b"\x00")[0].decode()
        
    return plainbuf


# extract XOR key
match = re.search(rb'\x48\x8d\x0d(?P<offset>[\s\S]{4})\x0f\xbe\x0c\x11', pe_loader.get_memory_mapped_image())
rip = match.start() + pe_loader.OPTIONAL_HEADER.ImageBase
offset = struct.unpack("I", match.group("offset"))[0]
instr_size = 7
xor_key_addr = rip + offset + instr_size
xor_key_rva = xor_key_addr - pe_loader.OPTIONAL_HEADER.ImageBase

# obfuscated const
xor_key_len = 17

xor_key = pe_loader.get_data(xor_key_rva, xor_key_len)
print(f"xor_key: {xor_key.hex()}")

print("\nascii strings:\n")
strs = []
# extracted from IDA for simplicity
ascii_rvas = [0x2a9e4, 0x2a418, 0x2a220, 0x2a3bb, 0x2ab60, 0x2a3e0, 0x2a0b0, 0x2aa20, 0x2aaa2, 0x2aa40, 0x2aad0, 0x2a2e5, 0x2aaa2, 0x2a220, 0x2a9c2, 0x2a0e4, 0x2ab08, 0x2a2cc, 0x2a357, 0x2a09a, 0x2aa53, 0x2a04f, 0x2ace3, 0x2a2ea, 0x2aaeb, 0x2a020, 0x2a390, 0x2a9ab, 0x2a3d1, 0x2a362, 0x2a3c9, 0x2a2f4, 0x2a22d, 0x2a25d, 0x2a044, 0x2aaf5, 0x2a144, 0x2aab6, 0x2a87b, 0x2a0ca, 0x2a9f0, 0x2a870, 0x2a05c, 0x2a138, 0x2a05c, 0x2acda, 0x2a9cd, 0x2ac40, 0x2ac40, 0x2a3bb, 0x2a850, 0x2a340, 0x2ab20, 0x2a070, 0x2aad0, 0x2a08b, 0x2a3e0, 0x2ad1e, 0x2ac74, 0x2ad28, 0x2a070, 0x2a240, 0x2a88a, 0x2a886, 0x2a82c, 0x2a835, 0x2a087, 0x2a030, 0x2a83a, 0x2aae8, 0x2aa9c, 0x2a0f0, 0x2a997, 0x2a9d9, 0x2aa34, 0x2a3b8, 0x2a9a8, 0x2aa9f, 0x2ad36, 0x2a14f, 0x2a101, 0x2a9dd, 0x2a9dd, 0x2a997, 0x2a2df, 0x2aaa2, 0x2a340, 0x2ac60, 0x2a990, 0x2ad00, 0x2ab20, 0x2a850, 0x2aaa2, 0x2ab00, 0x2abf0, 0x2a08b, 0x2aaa2, 0x2a0f3, 0x2a0d6, 0x2a89e, 0x2aaa2, 0x2a254, 0x2aaa2, 0x2acef, 0x2aaa2, 0x2a868, 0x2a891]
for i, rva in enumerate(ascii_rvas):
    # arbitrarily long length
    encbuf = pe_loader.get_data(rva, 256)
    plainbuf = decrypt_str(xor_key, encbuf)
    strs.append(plainbuf)
    
for i, plainbuf in enumerate(sorted(list(set(strs)))):
    print(f"{i}: {plainbuf}")
  
print("\nwide strings:\n")
strs = []
# extracted from IDA for simplicity
wide_rvas = [0x2a0de, 0x2a3a0, 0x2a02a, 0x2ab40, 0x2ac80, 0x2aa80, 0x2aa60, 0x2a300, 0x2a270, 0x2a270, 0x2a300, 0x2a1e0, 0x2a9b8, 0x2a083, 0x2a2d7, 0x2a0de, 0x2a083, 0x2aa92, 0x2a1c4, 0x2a110, 0x2a960, 0x2a110, 0x2aaac, 0x2abc0, 0x2a99a, 0x2a8b0]
for i, rva in enumerate(wide_rvas):
    # arbitrarily long length
    encbuf = pe_loader.get_data(rva, 256)
    plainbuf = decrypt_str(xor_key, encbuf, wide=True)
    strs.append(plainbuf)
    
for i, plainbuf in enumerate(sorted(list(set(strs)))):
    print(f"{i}: {plainbuf}")

xor_key: 8aa06ce696f5b4b96fdd6efa24af5393ab

ascii strings:

0: */*
1: .com
2: /post.php
3: ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/
4: Basic 
5: C:\Windows\System32\ntdll.dll
6: Connection: close

7: GET
8: Global\
9: HTTP/1.1
10: Local\
11: MSIMG32.dll
12: NtAllocateVirtualMemory
13: NtCreateThreadEx
14: NtCreateUserProcess
15: NtGetContextThread
16: NtOpenProcess
17: NtOpenSection
18: NtProtectVirtualMemory
19: NtReadVirtualMemory
20: NtResumeThread
21: NtSetContextThread
22: NtWriteVirtualMemory
23: POST
24: RtlGetVersion
25: RtlUserThreadStart
26: S-1-15
27: Software\Microsoft\
28: ThreadStart
29: User metrics
30: _ThreadStart@4
31: _alldiv
32: _aulldiv
33: _aullrem
34: advapi32.dll
35: aeiouy
36: bcdfghklmnpqrstvwxz
37: bcrypt.dll
38: br
39: cabinet.dll
40: crypt32.dll
41: div
42: dnsapi.dll
43: dxgi.dll
44: ftllib.dll
45: gdi32.dll
46: gdiplus.dll
47: h1
48: h2
49: h3
50: h4
51: h5
52: h6
53: hr
54: https://
55: imagehlp.dll
56: iphlpapi.dll
57: kernel32.

## Extract Version

The version can be found near the typical Zeus BinStorage 10003 (0x2713) CFGID marker.

Pseudocode:

    ...
    if ( v8 <= xor_0x19182A30(0x1B192D30) )   // 0x2010700
      v7 = 0x2010700;
    else
      v7 = v11;
    v11 = v7; 
    BinStorage::_addItem(p_binstorage, 10003, 0, &v11, 4u);// version
    ...
    
Disassembly:

    ...
    .text:00000000000E6A4F B8 00 07 01 02                                mov     eax, 2010700h
    ...
    .text:00000000000E6A6A BA 13 27 00 00                                mov     edx, 2713h
    ...

In [5]:
match = re.search(rb'\xb8(?P<version>[\s\S]{4})[\s\S]{1,24}\xba\x13\x27\x00\x00', pe_loader.get_memory_mapped_image())
version_bytes = match.group("version")
version = ".".join([str(b) for b in version_bytes[::-1]])
print(version)

2.1.7.0


## Decrypt BaseConfig

The BaseConfig is encrypted using RC4.

Pseudocode:

    __int64 __fastcall Core::getBaseConfig(__int64 a1)
    {
      __int64 v1; // rdx

      Mem::_copy(a1, &g_enc_base_config, 1020uLL);
      LOWORD(v1) = Str::_LengthA("lizrggmkfjdiulcdpjncwzyesmrvkeqr");
      return rc4("lizrggmkfjdiulcdpjncwzyesmrvkeqr", v1, a1, 1020u);
    }

Disassembly:

    ...
    .text:00000000000FED9E 48 8D 15 8B B6 00 00          lea     rdx, g_enc_base_config
    .text:00000000000FEDA5 41 B8 FC 03 00 00             mov     r8d, 3FCh
    .text:00000000000FEDAB E8 B0 A1 FF FF                call    Mem___copy
    ...
    .text:00000000000FEDBA 48 8D 0D 9F B3 00 00          lea     rcx, aLizrggmkfjdiul
    .text:00000000000FEDC1 E8 7A 61 00 00                call    Str___LengthA
    ...

In [6]:
match = re.search(rb'\x48\x8d\x15(?P<encbuf_offset>[\s\S]{4})\x41\xb8(?P<encbuf_len>[\s\S]{2}\x00\x00)\xe8[\s\S]{1,32}\x48\x8d\x0d(?P<rc4_key_offset>[\s\S]{4})', 
    pe_loader.get_memory_mapped_image())

rip = match.start() + pe_loader.OPTIONAL_HEADER.ImageBase
offset = struct.unpack("I", match.group("encbuf_offset"))[0]
instr_size = 7
encbuf_addr = rip + offset + instr_size
encbuf_rva = encbuf_addr - pe_loader.OPTIONAL_HEADER.ImageBase
encbuf_len = struct.unpack("I", match.group("encbuf_len"))[0]
encbuf = pe_loader.get_data(encbuf_rva, encbuf_len)

rip = match.start("rc4_key_offset") - 3 + pe_loader.OPTIONAL_HEADER.ImageBase
offset = struct.unpack("I", match.group("rc4_key_offset"))[0]
instr_size = 7
rc4_key_addr = rip + offset + instr_size
rc4_key_rva = rc4_key_addr - pe_loader.OPTIONAL_HEADER.ImageBase
rc4_key_chunk = pe_loader.get_data(rc4_key_rva, 64)
rc4_key = rc4_key_chunk.split(b"\x00")[0]

rc4 = ARC4.new(rc4_key)
base_config = rc4.decrypt(encbuf)
    
print(base_config.replace(b"\x00", b"."))

b'....Bing_Mod5............M1...................https://adslstickerhi.world...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................\x03...\x01...-----BEGIN PUBLIC KEY-----\nMIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQDKGAOWVkikqE7TyKIMtWI8dFsa\nleTaJNXMJNIPnRE/fGCzqrV+rtY3+ex4MCHEtq2Vwppthf0Rglv8OiWgKlerIN5P\n6NEyCfIsFYUMDfldQTF03VES8GBIvHq5SjlIz7lawuwfdjdEkaHfOmmu9srraftk\nI9gZO8WRQgY1uNdsXwIDAQAB\n-----END PUBLIC KEY-----\n..........

## Parse BaseConfig

In [7]:
defaultBotnet = base_config[0x4:0x4+32].split(b"\x00")[0].decode()
print(f"defaultBotnet: {defaultBotnet}\n")

campaign_id = base_config[0x19:0x19+32].split(b"\x00")[0].decode()
print(f"campaign_id: {campaign_id}\n")

url_chunk = base_config[0x2e:0x2e + 10 * 65]
urls = [u.decode().replace("http", "hxxp") for u in url_chunk.split(b"\x00")[0].split(b"\r\n")]
print(f"urls: {urls}\n")

flag1 = struct.unpack("I", base_config[0x2b8:0x2b8+4])[0]
print(f"flag1: {flag1}\n")

flag2 = struct.unpack("I", base_config[0x2bc:0x2bc+4])[0]
print(f"flag2: {flag2}\n")

rsa_public_key = base_config[0x2c0:0x2c0+512].split(b"\x00")[0].decode()
print(f"rsa_public_key: {rsa_public_key}\n")

defaultBotnet: Bing_Mod5

campaign_id: M1

urls: ['hxxps://adslstickerhi.world']

flag1: 3

flag2: 1

rsa_public_key: -----BEGIN PUBLIC KEY-----
MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQDKGAOWVkikqE7TyKIMtWI8dFsa
leTaJNXMJNIPnRE/fGCzqrV+rtY3+ex4MCHEtq2Vwppthf0Rglv8OiWgKlerIN5P
6NEyCfIsFYUMDfldQTF03VES8GBIvHq5SjlIz7lawuwfdjdEkaHfOmmu9srraftk
I9gZO8WRQgY1uNdsXwIDAQAB
-----END PUBLIC KEY-----




## Domain Generation Algorithm (DGA)

If needed, a backup DGA is used.

Pseudocode:

    ...
    time = Time::_getLocalTime_start_of_the_day();
    num_dga_domains = xor_0xCD6A5E6C19182A30(0xCD6A5E6C19182A10uLL);// 32
    dga(time, num_dga_domains, s24_dga_domains);
    ...

Pseudocode 2:

    __int64 __fastcall dga(unsigned int time, unsigned __int64 num_domains, s24 *s24_domains)
    {   
      __int64 result; // rax 
      __int64 tld; // rax 
      unsigned int math2; // [rsp+34h] [rbp-64h]
      int math1; // [rsp+38h] [rbp-60h]
      int math3; // [rsp+3Ch] [rbp-5Ch]
      s24 s24_tld; // [rsp+40h] [rbp-58h] BYREF
      char chr; // [rsp+5Bh] [rbp-3Dh]
      int j; // [rsp+5Ch] [rbp-3Ch]
      s24 s24_domain; // [rsp+60h] [rbp-38h] BYREF
      int i; // [rsp+78h] [rbp-20h]
      char v17[5]; // [rsp+93h] [rbp-5h] BYREF

      for ( i = 0; ; ++i )
      {   
        result = i;
        if ( i >= num_domains )
          break;
        s24_alloc_12(&s24_domain);
        for ( j = 0; j < 20; ++j )
        {   
          chr = time % 25 + 'a';
          s24_append_chr(&s24_domain, chr);
          time = a2_plus_a1(time, chr);
          math1 = time << 8;
          math2 = ~(time >> xor_0x19182A30(0x19182A28)) | 0xFFFFFF00;// 0x18
          math3 = (xor_0x19182A30(0xA168E34D) | 0x478F3682) & ~math2;// 0xb870c97d
                                                    // 0xffffffff
          time = math3 ^ math1 | a2_and_a1(math1, math3);
        }   
        tld = CryptedStrings::_getA(&byte_10A2E5, v17);// b'.com'
        w_s24_copy_in(&s24_tld, tld);
        s24_append_0(&s24_domain, &s24_tld);
        s24_dtor_0(&s24_tld);
        s24_array_24_add_entry(s24_domains, &s24_domain);
        s24_dtor_0(&s24_domain);
      }   
      return result;
    }  

In [8]:
def get_dga_domains(seed, num_domains):
    domains = []
    for _ in range(num_domains):
        domain = ""
        # domains are 20 chars long
        for _ in range(20):
            char = seed % 25 + ord('a')
            domain += chr(char)

            seed = char + seed 
            math1 = (seed << 8) & 0xffffffff
            math2 = ~(seed >> 0x18) | 0xffffff00
            math3 = ~math2 & 0xffffffff
            seed = math3 ^ math1 | (math3 & math1)

        domain += ".com"
        domains.append(domain)

    return domains


dt = datetime.datetime(2024, 4, 2, 20, 0)
seed = int(time.mktime(dt.timetuple()))

domains = get_dga_domains(seed, 32)
for domain in domains:
    print(domain)

arvokhvoacbjurqtfcge.com
fcqeprqefrbjurqtfcge.com
fcqeprqefrbjurvypcbt.com
kmltahvtfrbjurvypcbt.com
kmqykhqjkcbepcqtawqy.com
uhgeucgouwqjawgykrlj.com
ucvokhqjkcbephveumbo.com
fwltucltfrgtphveumbo.com
fcqeprqefmvyacqopcge.com
fcqeprqefrbjurvypwvj.com
pwlokhvtfrbjurvypcbt.com
kmqykclypmvtphbopcgy.com
uhgeucgouwqjurveumbo.com
arbyfwbypmvyawleumbo.com
fwltucltfrgtphbjfhqt.com
prgeprqefmvyacqopcge.com
fcvjamgjphqjurvykrlo.com
arbtucltkcbekwgtfclj.com
pwlokhbypmvtphbjkmgy.com
uhgeuwgjkcbekwleuhve.com
fcvjfrvouwqjawleumbo.com
arbyfrvouwqjawleumbo.com
fwqyfwbyurlyawlearlj.com
ucbtucltkwvtphbjkmgt.com
prgeuwbyurlyacqouhve.com
fwqykcqyurlyfhgouhve.com
fcvjfrvouwqjawlypwbo.com
fwqtahbypmbyfhgtarvy.com
amveuwgjphqepcvykrqo.com
fwqtamloacgjawlearqo.com
fwqtfrvjphqephgouhvy.com
amveahbtkwvokwlypwbj.com


## Command and Control

Command and Control (C2) is via HTTP POST requests. POST data contains encrypted Zeus BinStorage data structures. We're going to look at the following types of requests: 

* Hello
* Get module
* Get DynamicConfig

The examples below are based on a combination of a sample with a live C2 (not the analyzed sample) and a C2 emulator.

## Command and Control: Encryption

A 32-byte random RC4 key is generated. The RC4 key is PKCS#1 v1.5 encrypted using the public RSA key from the BaseConfig above. Data is encrypted using Zeus' VisualEncrypt algorithm and then RC4 using the random key. POST data is formatted like:

&lt;RSA encrypted RC4 key (128-bytes)&gt;&lt;encrypted data&gt;

Pseudocode:

      ...
      s24_ctor(s24_rand_key_then_enc_rand_key);
      s24_rand32(s24_rand_key_then_enc_rand_key);
      rand_key_len = s24_get_len(s24_rand_key_then_enc_rand_key);
      rand_key_buf = s24_get_buf(s24_rand_key_then_enc_rand_key);
      Crypt::_rc4Init(rand_key_buf, rand_key_len, rand_key_rc4_S);
      if ( (base_config_rsa_public_key_encrypt(s24_rand_key_then_enc_rand_key) & 1) != 0 )
      {
        w_Mem::_copy(rand_key_rc4_S_cp, rand_key_rc4_S, 0x102uLL);
        s24_ctor_copy_in(&s24_post_data, s24_rand_key_then_enc_rand_key);
        s24_append(&s24_post_data, a_data, a_data_len);
        post_data_len = s24_get_len(&s24_post_data);
        actual_data_len = post_data_len - s24_get_len(s24_rand_key_then_enc_rand_key);
        post_data_buf = s24_get_buf(&s24_post_data);
        actual_data_buf = s24_get_len(s24_rand_key_then_enc_rand_key) + post_data_buf;
        Crypt::_visualEncrypt(actual_data_buf, actual_data_len);
        Crypt::_rc4(actual_data_buf, actual_data_len, rand_key_rc4_S_cp);
        ...
       
Response data is encrypted similarly.

In [9]:
def rc4(key, in_buf):
    return rc4_keystate(rc4_ksa(key), in_buf)


def rc4_ksa(key):
    S = []
    for i in range(256):
        S.append(i)
    S.append(0)
    S.append(0)

    j = 0 
    for i in range(256):
        j = (j + S[i] + key[i % len(key)]) % 256 
        S[i], S[j] = S[j], S[i]

    S = bytes(S)

    return S


def rc4_keystate(S, in_buf):
    out_buf = b""

    i = S[256]
    j = S[257]
    
    for in_byte in in_buf:
        (i, j, S, K) = rc4_prga(i, j, S)
        out_byte = in_byte ^ K
        out_buf += struct.pack("B", out_byte)

    return out_buf


def rc4_prga(i, j, S):
    i = (i + 1) % 256
    j = (j + S[i]) % 256

    S = bytearray(S)
    S[i], S[j] = S[j], S[i]
    S = bytes(S)

    K = S[(S[i] + S[j]) % 256]

    return (i, j, S, K)


def visual_decrypt(enc_buf):
    buf = bytearray(enc_buf)

    for i in range(len(buf)-1, 0, -1):
        buf[i] ^= buf[i-1]

    plain_buf = bytes(buf)

    return plain_buf


# example RSA encryption of random RC4 key
rand_rc4_key = b'\x9eZ\xf8D\x80\x03\xb4p\xb3\x04g\xb5\xa5\xc9\xe4Y[\x1cZ\xa1\x185C\xb2sVh~\xe2\x80r?'

rsa_key = RSA.importKey(rsa_public_key)
rsa_obj = PKCS1_v1_5.new(rsa_key)
        
enc_rand_rc4_key = rsa_obj.encrypt(rand_rc4_key)
# swap for CryptoAPI little endian
enc_rand_rc4_key_rev = enc_rand_rc4_key[::-1]

print(f"{len(enc_rand_rc4_key_rev)=}")
print(f"{enc_rand_rc4_key_rev=}")

len(enc_rand_rc4_key_rev)=128
enc_rand_rc4_key_rev=b'\x86,`\xa2\xb5\xfa+\x1f\xe1cY\xb6\xf8\xa7\x87?\x90\x80=\x84\x90\x93\x8d\x89\x1d\x18\xc6}\x95\xb6\xd8\xd5\x98\x99\xe0K\xa1\xdfB\xed\xbf-\xe9E\xb5\x9d\xd0\x9c\xdc\xa4u@\xc2\x88\x1a$\x93\x17\x9fl0\r\xd7\xa8\x16\x11\xe9N\xac\xb3\xb0\x18\x8b`\xbe\xfd\x91\xb8c\x89\x1cW\xa3\xf7\xc2\xa9\xc8\r\x18\xbb\xc8f\x80{\x97\xfba\xf9\x8d\xd3\x8d\xde\x02\x16\x8e\xb5\xc6\xcbzk\x904RUj\xc9~\x97&\x97\x12\x8c}\x90\x19*J!'


## Command and Control: Basic BinStorage

The requests below all start with the same BinStorage items:

* 10002 (0x2712) - botnet
* 10025 (0x2729) - campaign ID
* 10001 (0x2711) - bot ID
* 10003 (0x2713) - malware version

In [10]:
def parse_binstorage_header(binstorage):
    header = {
        "randData": binstorage[0:20],
        "size": struct.unpack("I", binstorage[20:24])[0],
        "flags": struct.unpack("I", binstorage[24:28])[0],
        "count": struct.unpack("I", binstorage[28:32])[0],
        "md5Hash": binstorage[32:48],
    }
        
    return header


def parse_binstorage_sections(section_data, num_sections):
    CFGIDS = {
    }
    
    sections = []
    offset = 0
    for i in range(num_sections):
        section = {}
        
        id_ = struct.unpack("I", section_data[offset:offset+4])[0]
        if int(id_) in CFGIDS:
            section["id"] = f'{CFGIDS[int(id_)]} ({id_})'
        else:
            section["id"] = id_
            
        section["flags"] = struct.unpack("I", section_data[offset+4:offset+8])[0]
        section["size"] = struct.unpack("I", section_data[offset+8:offset+12])[0]
        section["realSize"] = struct.unpack("I", section_data[offset+12:offset+16])[0]
        # additional parsing of data is left as an exercise
        section["data"] = section_data[offset+16:offset+16+section["size"]]
        
        offset += 4 + 4 + 4 + 4 + section["size"]
        
        sections.append(section)
        
    return sections

## Command and Control: Hello Request

![](hello.png)

The "hello" request contains the following BinStorage item:

* 10006 (0x2716) - unknown hardcoded `1`.

The response is an empty BinStorage.

In [11]:
post_data = b'\xe0\xabM0h\xfa\xf7M\x19\xbe(5\x87]ri\xba\xce\xbe\xaf\x97\x84\xfdax\x12~=\xff\xb4\xa5\x16\x15\xc1\xaf\x99\xfa\xbd\xb4\x9e\xc8-1\xcat\x82\x1a \xd3\xeb\x8d.\xfa\xb0|\xf5\x81\x8e\xb8\x01SnF\xb9/\xbc(\xeao\x02\x9d0\x88\x8b"t\x19\xa9\xde\xdf\x1d\x89\x7f\x88\x0b[\xb0pNP\xea\x87\xfc\x11\xe2\xbc8XO\x16gkk\xe4#\x11\rtu\xa70H7v\x92\xb5\x17\\\xfcg\xaf\x03;X6*\xc7\x87\xec9\x02\x0f\x039N\xfc\x83\xcd\xefF\xd3\x0fF\x7f\x94\xed\xd6\xf8\x84>\xf2s\xff\xe2\x90\xd3\x8f\xf5U\x83X>tK\xf3\xcd\x9cbV+?\x9b\xab\x15\x94\x12\x81W\x83\x86\x7f\xe1u\x88\xb3\x8d\x9e\xc8\x86\x9bg@l\xbd \xc1\xc2\xadHL\x06\x01+)\x14\x85\xdc\xebvl \x8cKr\x90\xc1\xda\xa8\x8f\x1eY:^\xdaJ\xa0 \xe8\xef+u\x0e@\xf3\xd3SL\xf1\x83z\xd2\x04`Ls\x80D\x94%\x95e\x85+\xa3*#\xa8-\x84y{\x0ca\xb0\xde\x80\xf3\x1f\t\xe3\xad5\xde;W5\xfe\xee9\xb3\xbb\xac\xdb\xf2\x98\xd8re\xe2\xff\x82\x19\xd796=\xc8"\x06'
encrypted_request = post_data[128:]

round1 = rc4(rand_rc4_key, encrypted_request)
binstorage = visual_decrypt(round1)

binstorage_header = parse_binstorage_header(binstorage)
print("request binstorage_header:\n")
pprint.pprint(binstorage_header)

binstorage_sections = parse_binstorage_sections(binstorage[48:], binstorage_header["count"])
print()
print("request binstorage_sections:\n")
pprint.pprint(binstorage_sections)

response = b'\x99\xb3kw\xe7\x80"hS\x7f\xff\xc3P\x17\xdb\x97\x10\xda\x15\xb4#\x99U\xd4XE7t,V\xf6 \xc1\xc4\x8d\xb5\x1a\xa8r\x8d\xd0\x0bv\xf3\xc6\x84Z\\\xdd,\xf8\xfd\x04\x9a\x0e\xf3\xcb\xf5\xe6\xb0\xfd\xe0\x1c;'
round1 = rc4(rand_rc4_key, response)
binstorage = visual_decrypt(round1)

binstorage_header = parse_binstorage_header(binstorage)
print("\nresponse binstorage_header:\n")
pprint.pprint(binstorage_header)

binstorage_sections = parse_binstorage_sections(binstorage[48:], binstorage_header["count"])
print()
print("response binstorage_sections:\n")
pprint.pprint(binstorage_sections)

request binstorage_header:

{'count': 5,
 'flags': 0,
 'md5Hash': b'p\x84\x101\xf3\xc7r\xbe\x10t\x1c\x0f&\xb4G%',
 'randData': b'=\x10.fG\xcf\x8c\xaa\xe0e,\xa8\xd7\xadE"u\xd3}\x9a',
 'size': 171}

request binstorage_sections:

[{'data': b'Zep', 'flags': 0, 'id': 10002, 'realSize': 3, 'size': 3},
 {'data': b'M2', 'flags': 0, 'id': 10025, 'realSize': 2, 'size': 2},
 {'data': b'DESKTOP-DBHLUTB_lache_640FBDD2',
  'flags': 0,
  'id': 10001,
  'realSize': 30,
  'size': 30},
 {'data': b'\x00\x04\x06\x02',
  'flags': 0,
  'id': 10003,
  'realSize': 4,
  'size': 4},
 {'data': b'\x01\x00\x00\x00',
  'flags': 0,
  'id': 10006,
  'realSize': 4,
  'size': 4}]

response binstorage_header:

{'count': 1,
 'flags': 0,
 'md5Hash': b'J\xe7\x136\xe4K\xf9\xbfy\xd2u.#H\x18\xa5',
 'randData': b'H\xef\xcdw\xdb\x92YR\xa4\x07\x8e=\xd16\xc0W\x19`\x89\x15',
 'size': 64}

response binstorage_sections:

[{'data': b'', 'flags': 0, 'id': 0, 'realSize': 0, 'size': 0}]


## Command and Control: Get Main Module Request

![](main_module.png)

The loader component retrieves one module (ID 2) which is the main module (Client64.dll). Its request contains the following BinStorage items:

* 12000 (0x2ee0) - module ID
* 10038 (0x2736) - unknown hardcoded `0`

The response is plaintext and contains a 24-byte header followed by the module. The header contains:

* Module ID
* Module version
* Unknown DWORD
* Unknown DWORD
* Module CRC32
* Module length

The example main module is on [MalwareBazaar](https://bazaar.abuse.ch/sample/f20c687c0509b874f2b2a141475415539ee3bed2c5b4eac77dcf5a7823286573/).

In [12]:
post_data = b'\xbd24\xa8=5\xd9\x11nP\xce\x92_rh\x17\xc5tE\x8c\xbc!\x00\x13$\xbe\xf0\xf1\xe0\x08\x94\x0c\xf6o\xcbt\xdb\x96vn\x97V\x01\xb3LD?\xb5*\xba?\x08NO,\xbb\x11\xb8V\xc6\x14\x1e\x85\xf3Re\xee#\xb62.\x98\xd0\x15W\x8d_+\xba\x87\x07O\xeeD_\x00\xc0\x03\x17\xd4\x9b\xda/r\x1a\xf4\x99\x06\x0f\xf6\xbd7\xce\xfe,e\\\x81I\xa9\xd1\xa9vL`\x83\xeb\xd3\xa4`\xcd\xf2O^@r\xba;\xab&5\xa4\x9aq\xdf\x96\xdbr\x8d\x02\xac\xa8 r\xe9\x7f\xa8\xae\xc6|\xb01\xbd\xa0\xd2\x91\xce\xb4\x14\xc2Zl\xc9 g\xa8\x81\x11\x8eK\x9en\xf8\x8e\xac\xdeM\x9bOJ\xb3-\xb9D\x7fAR\x04JW\xab\x8c\xa0q\xec\r\x0ea\x84\x80\xca\xcd\xe7\xe5\xd8I\x10\'\xba\xa0\xec@\x87\xbe\\\r\x16dC\xd2\x95\xf6\x92\x16\x86l\xec$#\xe7\xb9\xc2\x8c?\x1f\x9f\x80=O\xb6\x1e\xc8\xac\x80\xbfL\x88X\xe9Y\xa9I\xe7o\xe6\xefd\xe1H\xb5\xb7\xc0\xad|\x12L?\xd3\xc5/a\xf9\x12\xf7\x9b\xf92"\xf5\x89\x88\x9f\xe8\xc1\xab\xebAV\xd1\xcc\xb1*\xe4\n\x05\r\xf8\x126C\x01\x11\xcf\xa9]\x07\xa2\x99\xcb4\x19\x83\x95\x10Hk\xe9\x08\xf9'
encrypted_request = post_data[128:]

round1 = rc4(rand_rc4_key, encrypted_request)
binstorage = visual_decrypt(round1)

binstorage_header = parse_binstorage_header(binstorage)
print("request binstorage_header:\n")
pprint.pprint(binstorage_header)

binstorage_sections = parse_binstorage_sections(binstorage[48:], binstorage_header["count"])
print()
print("request binstorage_sections:\n")
pprint.pprint(binstorage_sections)

print("\nresponse:\n")

response = b'\x02\x00\x00\x00\x00\x00\x03\x02\x00\x00\x00\x00"E\xa8e\x87\xb5:\xc4\x00\xb6\x04\x00MZ\x90\x00\x03\x00\x00\x00\x04\x00\x00\x00\xff\xff\x00\x00\xb8\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x84\x04\x00\xf0\x8c\x04\x00h\xa8\x04\x00\xd8\x00\x00\x00\x0e\x1f\xba\x0e\x00\xb4\t\xcd!\xb8\x01L\xcd!This program cannot be run'

module_header = response[0:24]
truncated_module = response[24:]

module_id = struct.unpack("I", module_header[0:4])[0]
print(f"{module_id=}")

module_version = struct.unpack("I", module_header[4:8])[0]
print(f"module_version=0x{module_version:x}")

module_unknown1 = struct.unpack("I", module_header[8:12])[0]
print(f"{module_unknown1=}")

module_unknown2 = struct.unpack("I", module_header[12:16])[0]
print(f"{module_unknown2=}")

module_crc32 = struct.unpack("I", module_header[16:20])[0]
print(f"module_crc32=0x{module_crc32:x}")

module_length = struct.unpack("I", module_header[20:24])[0]
print(f"{module_length=}")

print(f"\n{truncated_module=}...")

# Load the main module
fp = open("f20c687c0509b874f2b2a141475415539ee3bed2c5b4eac77dcf5a7823286573.bin", "rb")
main = fp.read()
fp.close()

pe_main = pefile.PE(data=main)

request binstorage_header:

{'count': 6,
 'flags': 0,
 'md5Hash': b'3\xd4\xff\xe7\x0c6\n\xd0\xbb\xcc\xdd[\x80|\xe4\xd1',
 'randData': b'zH\x06\xfau\x1eUQ\xd2\x82\xf1\x8e\xecu\x84I\x05<\x91\xb2',
 'size': 191}

request binstorage_sections:

[{'data': b'Zep', 'flags': 0, 'id': 10002, 'realSize': 3, 'size': 3},
 {'data': b'M2', 'flags': 0, 'id': 10025, 'realSize': 2, 'size': 2},
 {'data': b'DESKTOP-DBHLUTB_lache_640FBDD2',
  'flags': 0,
  'id': 10001,
  'realSize': 30,
  'size': 30},
 {'data': b'\x00\x04\x06\x02',
  'flags': 0,
  'id': 10003,
  'realSize': 4,
  'size': 4},
 {'data': b'\x02\x00\x00\x00',
  'flags': 0,
  'id': 12000,
  'realSize': 4,
  'size': 4},
 {'data': b'\x00\x00\x00\x00',
  'flags': 0,
  'id': 10038,
  'realSize': 4,
  'size': 4}]

response:

module_id=2
module_version=0x2030000
module_unknown1=0
module_unknown2=1705526562
module_crc32=0xc43ab587
module_length=308736

truncated_module=b'MZ\x90\x00\x03\x00\x00\x00\x04\x00\x00\x00\xff\xff\x00\x00\xb8\x00\x00\x00\x00\x00

## Command and Control: Other Modules

Other module IDs include:

* 0 - unknown (maybe nothing)
* 1 - Client32.dll
* 2 - Client64.dll
* 3 - HttpGrabber.dll
* 4 - SoftwareGrabber.dll
* 5 - Vnc32.dll
* 6 - Vnc64.dll
* 7 - ftp64.dll

## Decrypt Strings (Main Module)

In [13]:
# extract XOR key
match = re.search(rb'\x48\x8d\x0d(?P<offset>[\s\S]{4})\x0f\xbe\x0c\x11', pe_main.get_memory_mapped_image())
rip = match.start() + pe_main.OPTIONAL_HEADER.ImageBase
offset = struct.unpack("I", match.group("offset"))[0]
instr_size = 7
xor_key_addr = rip + offset + instr_size
xor_key_rva = xor_key_addr - pe_main.OPTIONAL_HEADER.ImageBase

# obfuscated const
xor_key_len = 17

xor_key = pe_main.get_data(xor_key_rva, xor_key_len)
print(f"xor_key: {xor_key.hex()}")

print("\nascii strings:\n")
strs = []
# extracted from IDA for simplicity
ascii_rvas = [0x495d0, 0x495d0, 0x497e0, 0x497e0, 0x49760, 0x49760, 0x4abb0, 0x4abb0, 0x48370, 0x48370, 0x49a00, 0x49a00, 0x48300, 0x48300, 0x49059, 0x49059, 0x4a211, 0x4a211, 0x49bc0, 0x49bc0, 0x4aed0, 0x4aed0, 0x483ca, 0x483ca, 0x4ae80, 0x4ae80, 0x48730, 0x48730, 0x4866c, 0x4866c, 0x4ac30, 0x4ac30, 0x48626, 0x48626, 0x49005, 0x49005, 0x497ff, 0x497ff, 0x48632, 0x48632, 0x4856f, 0x4856f, 0x482b6, 0x482b6, 0x4ab9c, 0x4ab9c, 0x49da1, 0x49da1, 0x4a650, 0x4a650, 0x486f0, 0x486f0, 0x48280, 0x48280, 0x48100, 0x48100, 0x48fc0, 0x48fc0, 0x4abd0, 0x4abd0, 0x4aae0, 0x4aae0, 0x48120, 0x48120, 0x4aa10, 0x4aa10, 0x4a5d8, 0x4a5d8, 0x48299, 0x48299, 0x498da, 0x498da, 0x4ad90, 0x4ad90, 0x4ac90, 0x4ac90, 0x4a390, 0x4a390, 0x48e80, 0x48e80, 0x4ad60, 0x4ad60, 0x48d34, 0x48d34, 0x49e50, 0x49e50, 0x48170, 0x48170, 0x4a670, 0x4a670, 0x4a750, 0x4a750, 0x494bc, 0x494bc, 0x48e60, 0x48e60, 0x48e99, 0x48e99, 0x48224, 0x48224, 0x49987, 0x49987, 0x4a9fe, 0x4a9fe, 0x4a730, 0x4a730, 0x4ac90, 0x4ac90, 0x49090, 0x49090, 0x4ab20, 0x4ab20, 0x48500, 0x48500, 0x4a6a0, 0x4a6a0, 0x48750, 0x48750, 0x49d30, 0x49d30, 0x49730, 0x49730, 0x487a0, 0x487a0, 0x49e60, 0x49e60, 0x49ce0, 0x49ce0, 0x4adb0, 0x4adb0, 0x4a530, 0x4a530, 0x49db0, 0x49db0, 0x4a220, 0x4a220, 0x48f10, 0x48f10, 0x48550, 0x48550, 0x4a430, 0x4a430, 0x4aac0, 0x4aac0, 0x49b80, 0x49b80, 0x4a970, 0x4a970, 0x49c30, 0x49c30, 0x48580, 0x48580, 0x49550, 0x49550, 0x4a750, 0x4a750, 0x4aaae, 0x4aaae, 0x48b9a, 0x48b9a, 0x4a340, 0x4a340, 0x482dd, 0x482dd, 0x488f0, 0x488f0, 0x484d0, 0x484d0, 0x482dd, 0x482dd, 0x48710, 0x48710, 0x49b68, 0x49b68, 0x49ec2, 0x49ec2, 0x4998b, 0x4998b, 0x4a7e8, 0x4a7e8, 0x4a352, 0x4a352, 0x48794, 0x48794, 0x48dd0, 0x48dd0, 0x4a240, 0x4a240, 0x495f0, 0x495f0, 0x483a0, 0x483a0, 0x49810, 0x49810, 0x495d0, 0x495d0, 0x48ce0, 0x48ce0, 0x48c90, 0x48c90, 0x48ce0, 0x48ce0, 0x48c90, 0x48c90, 0x49e44, 0x49e44, 0x485a4, 0x485a4, 0x48387, 0x48387, 0x4ab4d, 0x4ab4d, 0x48cf1, 0x48cf1, 0x483da, 0x483da, 0x4824c, 0x4824c, 0x48c1e, 0x48c1e, 0x49c58, 0x49c58, 0x49f9c, 0x49f9c, 0x4ab14, 0x4ab14, 0x480d2, 0x480d2, 0x4a930, 0x4a930, 0x4a45c, 0x4a45c, 0x4891f, 0x4891f, 0x4853c, 0x4853c, 0x4a6cb, 0x4a6cb, 0x480c0, 0x480c0, 0x4970e, 0x4970e, 0x4a528, 0x4a528, 0x4aeca, 0x4aeca, 0x4a7ed, 0x4a7ed, 0x4ad83, 0x4ad83, 0x4a720, 0x4a720, 0x48230, 0x48230, 0x49020, 0x49020, 0x48e80, 0x48e80, 0x482bb, 0x482bb, 0x49979, 0x49979, 0x4ad46, 0x4ad46, 0x49aa0, 0x49aa0, 0x4aea0, 0x4aea0, 0x482bb, 0x482bb, 0x48299, 0x48299, 0x4a5e2, 0x4a5e2, 0x485b0, 0x485b0, 0x4a376, 0x4a376, 0x4aee8, 0x4aee8, 0x497f4, 0x497f4, 0x48d28, 0x48d28, 0x4ae8a, 0x4ae8a, 0x4971f, 0x4971f, 0x4aebe, 0x4aebe, 0x482c5, 0x482c5, 0x4a912, 0x4a912, 0x4a966, 0x4a966, 0x484eb, 0x484eb, 0x4a936, 0x4a936, 0x4a91c, 0x4a91c, 0x49ba6, 0x49ba6, 0x490af, 0x490af, 0x49ad4, 0x49ad4, 0x4849c, 0x4849c, 0x4905f, 0x4905f, 0x494c1, 0x494c1, 0x48bb0, 0x48bb0, 0x4ad3b, 0x4ad3b, 0x4ad2e, 0x4ad2e, 0x48020, 0x48020, 0x48660, 0x48660, 0x4818f, 0x4818f, 0x48541, 0x48541, 0x4899c, 0x4899c, 0x48426, 0x48426, 0x4899c, 0x4899c, 0x4aba2, 0x4aba2, 0x49832, 0x49832, 0x48b56, 0x48b56, 0x4983e, 0x4983e, 0x4a929, 0x4a929, 0x494cc, 0x494cc, 0x49070, 0x49070, 0x4ad7d, 0x4ad7d, 0x4819a, 0x4819a, 0x49b9d, 0x49b9d, 0x49b3c, 0x49b3c, 0x4a9fa, 0x4a9fa, 0x49ed0, 0x49ed0, 0x482bb, 0x482bb, 0x4900b, 0x4900b, 0x482bb, 0x482bb, 0x48269, 0x48269, 0x482bb, 0x482bb, 0x482d4, 0x482d4, 0x495d0, 0x495d0, 0x494cc, 0x494cc, 0x49b50, 0x49b50, 0x49aa0, 0x49aa0, 0x480e0, 0x480e0, 0x48ef0, 0x48ef0, 0x4a570, 0x4a570, 0x48d3c, 0x48d3c, 0x4a590, 0x4a590, 0x4a570, 0x4a570, 0x480e0, 0x480e0, 0x49530, 0x49530, 0x48360, 0x48360, 0x4a590, 0x4a590, 0x49d60, 0x49d60, 0x494f3, 0x494f3, 0x494f3, 0x494f3, 0x49a67, 0x49a67, 0x49700, 0x49700, 0x48907, 0x48907, 0x48789, 0x48789, 0x482bb, 0x482bb, 0x499bb, 0x499bb, 0x495d0, 0x495d0, 0x4a5d8, 0x4a5d8, 0x48ef0, 0x48ef0, 0x49ac0, 0x49ac0, 0x4aef3, 0x4aef3, 0x48914, 0x48914, 0x48f8a, 0x48f8a, 0x49cc0, 0x49cc0, 0x49960, 0x49960, 0x48250, 0x48250, 0x48ff0, 0x48ff0, 0x48bc0, 0x48bc0, 0x4aa50, 0x4aa50, 0x49a80, 0x49a80, 0x4aa90, 0x4aa90, 0x4a360, 0x4a360, 0x4aa70, 0x4aa70, 0x498c0, 0x498c0, 0x49ea0, 0x49ea0, 0x4ad4a, 0x4ad4a, 0x497d2, 0x497d2, 0x499fa, 0x499fa, 0x4a338, 0x4a338, 0x4abca, 0x4abca, 0x49e40, 0x49e40, 0x48b5e, 0x48b5e, 0x485ee, 0x485ee, 0x4a5a1, 0x4a5a1, 0x4a7f1, 0x4a7f1, 0x49a5a, 0x49a5a, 0x49fe8, 0x49fe8, 0x4a356, 0x4a356, 0x4a383, 0x4a383, 0x4a383, 0x4a383, 0x4abca, 0x4abca, 0x48c7e, 0x48c7e, 0x482bb, 0x482bb, 0x49a52, 0x49a52, 0x4ab08, 0x4ab08, 0x4aca1, 0x4aca1, 0x4a352, 0x4a352, 0x48794, 0x48794, 0x4ab19, 0x4ab19, 0x4a2b1, 0x4a2b1, 0x48d3c, 0x48d3c, 0x48c76, 0x48c76, 0x482bb, 0x482bb, 0x494e0, 0x494e0, 0x49bdc, 0x49bdc, 0x49b50, 0x49b50, 0x48690, 0x48690, 0x48ea1, 0x48ea1]
for i, rva in enumerate(ascii_rvas):
    # arbitrarily long length
    encbuf = pe_main.get_data(rva, 256)
    plainbuf = decrypt_str(xor_key, encbuf)
    strs.append(plainbuf)
    
for i, plainbuf in enumerate(sorted(list(set(strs)))):
    print(f"{i}: {plainbuf}")
  
print("\nwide strings:\n")
strs = []
# extracted from IDA for simplicity
wide_rvas = [0x48df0, 0x48df0, 0x48df0, 0x48df0, 0x4a3d0, 0x4a3d0, 0x4ab60, 0x4ab60, 0x48d50, 0x48d50, 0x4867c, 0x4867c, 0x48930, 0x48930, 0x48030, 0x48030, 0x48640, 0x48640, 0x482a6, 0x482a6, 0x4a320, 0x4a320, 0x49850, 0x49850, 0x48c30, 0x48c30, 0x48f60, 0x48f60, 0x49dd0, 0x49dd0, 0x499a0, 0x499a0, 0x48b70, 0x48b70, 0x49d10, 0x49d10, 0x489b0, 0x489b0, 0x49bf0, 0x49bf0, 0x49a20, 0x49a20, 0x499d0, 0x499d0, 0x49ae0, 0x49ae0, 0x48950, 0x48950, 0x4acb0, 0x4acb0, 0x4ab60, 0x4ab60, 0x48600, 0x48600, 0x48df0, 0x48df0, 0x4a9a0, 0x4a9a0, 0x482e0, 0x482e0, 0x497c8, 0x497c8, 0x49780, 0x49780, 0x4a260, 0x4a260, 0x497c8, 0x497c8, 0x4a780, 0x4a780, 0x487ea, 0x487ea, 0x4ace0, 0x4ace0, 0x48cb0, 0x48cb0, 0x4adf0, 0x4adf0, 0x482e0, 0x482e0, 0x48a60, 0x48a60, 0x48800, 0x48800, 0x4a950, 0x4a950, 0x481c0, 0x481c0, 0x48f30, 0x48f30, 0x497c8, 0x497c8, 0x4a780, 0x4a780, 0x487ea, 0x487ea, 0x48df0, 0x48df0, 0x49a5d, 0x49a5d, 0x49a5d, 0x49a5d, 0x4a510, 0x4a510, 0x4a5f0, 0x4a5f0, 0x481a0, 0x481a0, 0x49df0, 0x49df0, 0x4a800, 0x4a800, 0x4a4d0, 0x4a4d0, 0x487d0, 0x487d0, 0x497c8, 0x497c8, 0x485c0, 0x485c0, 0x49711, 0x49711, 0x49ef0, 0x49ef0, 0x48930, 0x48930, 0x48440, 0x48440, 0x480a0, 0x480a0, 0x49500, 0x49500, 0x49fb0, 0x49fb0, 0x489d0, 0x489d0, 0x4a2c0, 0x4a2c0, 0x498a4, 0x498a4, 0x4a6e0, 0x4a6e0, 0x48eb0, 0x48eb0, 0x48330, 0x48330, 0x4a5b0, 0x4a5b0, 0x49ebc, 0x49ebc, 0x4ac50, 0x4ac50, 0x481f0, 0x481f0, 0x48d80, 0x48d80, 0x4ae50, 0x4ae50, 0x48be0, 0x48be0, 0x483e0, 0x483e0, 0x49ebc, 0x49ebc, 0x4a4bc, 0x4a4bc, 0x49fa1, 0x49fa1, 0x49610, 0x49610, 0x4a7b0, 0x4a7b0, 0x489f0, 0x489f0, 0x497c8, 0x497c8, 0x4a780, 0x4a780, 0x4a470, 0x4a470, 0x49c70, 0x49c70, 0x4abf0, 0x4abf0, 0x487d0, 0x487d0, 0x4a689, 0x4a689, 0x4a28a, 0x4a28a, 0x49516, 0x49516, 0x49680, 0x49680, 0x4a5b0, 0x4a5b0, 0x48f90, 0x48f90, 0x4a2f0, 0x4a2f0, 0x484b0, 0x484b0, 0x498ac, 0x498ac, 0x49fa1, 0x49fa1, 0x487ea, 0x487ea, 0x48d18, 0x48d18]
for i, rva in enumerate(wide_rvas):
    # arbitrarily long length
    encbuf = pe_main.get_data(rva, 256)
    plainbuf = decrypt_str(xor_key, encbuf, wide=True)
    strs.append(plainbuf)
    
for i, plainbuf in enumerate(sorted(list(set(strs)))):
    print(f"{i}: {plainbuf}")

xor_key: 1d30bc10f08a563176fecd7f4e0fe8c997

ascii strings:

0: 
[!] DLL terminated process.
1: 
[+] DLL loaded, the spawned process will destroy at the end of the shell session or you can force destroy via 'kill' command.
2: 
[-] Error: can't find export function.
3: 
[-] Internal Error: can't load DLL.
4: 
[-] Internal Error: unknown error.
5: 

6:  /? - show help

7:  bot_id - show bot id

8:  cd - display/change current directory 'cd [path]'

9:  dir - show contents of a directory 'dir [path]'

10:  exec - execute any executable file 'exec <program> [/a1]'

11:  exit - quit shell

12:  getdll - get dll 'getdll <dllname>'

13:  getm - get module 'getm <modname>'

14:  getsc - get shellcode 'getsc <filename>'

15:  kill - terminate process 'kill <pid>'

16:  rundll - run dll 'rundll <dllname[,dllexport]>'

17:  runm - run module 'runm <modname>'

18:  runsc - run shellcode 'runsc <filename> <-A64>|<-A32>'

19: %08X %08X
20: %s+0x%x
21: %u.%u.%u.%u
22: (unknown)
23: */*
24: .com
25: /

# Command and Control: Get DynamicConfig Request

The "get DynamicConfig" request contains the following BinStorage items:

* 10012 (0x271c) - Windows version
* 10023 (0x2727) - process integrity level
* 10024 (0x2728) - number of monitors
* 10016 (0x2720) - IPv4 address
* 10007 (0x2717) - unknown BaseConfig flag 1 value
* 10026 (0x272a) - MD5 of loader
* 10039 (0x2737) - process file name
* 10040 (0x2738) - current C2 URL 
* 10034 (0x2732) - domain forest name
* 10035 (0x2733) - domain dns name
* 10022 (0x2726) - unknown BaseConfig flag 2 value
* 10020 (0x2724) - process listing
* 10027 (0x272b) - timezone
* 29000 (0x7148) - GMT offset

In [14]:
# update RC4 key for example data
rand_rc4_key = b'4\xd9\xb8 \xfc\x0eb\x80H-$\xd2L\xf6}j<\xcf\x84F\tG\xef\xdaq\x0e4\x13E\xa4\x9d/'

post_data = b'\xb7\x92\xd8\x0e\x12xLD\xac<\xd5\xd1[W-"\x0fp\x83\x97Iy\x90m0n\xb2u\xcbT=\xa0e\xbf\x8f\xbc\x17\xea\x1c\xd1+NH\xd3$\x0f\xc6^,,\xbb\x00\x10\xd5\xcdde\xd91R\xd6\tYc\xed\t\x93\xb6\x17\xcb>_\xaa\x0e\xb6|Y\rZct\x99\x95\xd7\x9f\xe6\xbb!\xb1J\xca+}\xd5T$U_\x00L\xc2\xb8\xaaM\x1d=\x00\x03\x03\xa85\x87\x9b\x16`pBb\xb5\x18\xb2O\x8e\x1f\x85\xe6>\xab)(\xe3\x82h\xd1rU\x0c\xcdK\xeb\xd0J\xa9#\xbb\x19&\xd3\x96\x17\x18\xc69\xc9jb\x85\xd3\xc2\x12\x90"\x8d\xb1\xbag\xa11\xf5%ZX\x9c\x8c9\xfa\x0c\xa4\xf7O\xa4\xe5n\x9d\xc6L\x8b\x8f\xbe5\xfa\x9f\xce^7\xbbnKH\x93]\xbc\x19s\xa3\xed\xc7\xa5p\xfe\xb7\x8b>\x1d\xfd\xe6\xf7.$\x9b\xaf\xdd\xbf\x0f\xe0\xb2\x01\xb4\xd4\x06\xf2tpa\xdb\x00\x18\xff`K\x00\xe4\xc9\x14\xa5B)\x17\xea\x1bo\xe0^\xf2FHg\xbb\x91\xc9\x891[\x83\xfe\x07\xb1\xb0\x9c\xe66\xc3\x8e)\xfaJC\xae\x81\x1c\xb7q\xc0\xe3\x00\x8do\x93|\xec\x14jg\xf5w1\x93\xd6\x1e9p\xbb\xcb\x04\x93`H\x1c\xf5\x9cF\xae{\x7f\xc9\x9e\x0c\x8e\xbfx\xfcV_\xb2y\x8f\xec\xe9\xed\xe7\x1a\xddq\x05\x91\x1a\x07*x\xb7\x98"\xb7x\xa6\xc3\xfd\xb5Nv\xdfd \xbc\xa2}\xd2\x82\x89\t\xecO\xf0$\x96\xab\xfdr\xde\xe3\xc3l\x93\x90S\t\xcc\x99C\x00yw\xf8\x9f\x01\x0fk>\xeb`\xf6A\x92\xfd\xf5\xbf\x8f\xa2i\xfe\xe9?\xa7\xbev\x85\xca\x11\xae5j\xf1\xb8\xd9\xcfS\xccSj\xe0MhbrU>\x01\xe9\x9d"\x82:a\xce\xae\x8c~\xb3\xa08\xcf\xd7\xfd9\xa9^K\xc9\xb0u0O\x9d(\x94\xd9\x83\x01\xb6\rEu\x86lnN\x1euq\x99\xb9\xddI\x15\xee\xe7\xe1\xa5!|SD\xda\x8a\x05\xcbd\x84\x1cVF\xe2\x84E!\xa6{,\x99U\x0c\xcd\xe2\x82\xdc%\'\x87\xd8\x9e\xbfM\n\x88%\xb9\xe7\xbcRX\xe2c\xa9\xcfTM\xcf*\xfb\xe7r\xec\x07\xa2wB\xcdn\xc3\xfd\x828\xd3\xa9\xa5t"N\xb6n\xe3E\x98,a5u8\xde\xe1\xa6\x1cNj\xef\xb1\xe6]\xa2\t\xa1\xd0\xa7\x167\x00\xc3\x9e\x86U\x18\xd6\xff\x84*\xb6+a\x9b?\xcd\x8e{/\xb6\xdf\xf0\xe3\x00?:\xed\xbbz\'\'\x1b\x82A\xf9F\xf3p\x90\x8b\xb72\tn\xcf#f\xd6\xd5\xb7x\xe2\x9c\xeb\x1c\xa6\xe8\xe4=,\xe7\xa9\xb8X\xd1\xd8@5\xe0`\x9b\xc8A\x95&\x08Q\xa5\xf3\x89\xc7\xf4\xea\x12\x1a\xe2\xbf\xebO2r%<\x94\x9e\xb4Z\xa9\xb2E\x1f\xbf4\x8a}\xd0PO\xf8\xa8(>\xe2\x8d\xc4\x8a\xa3\xd6\xd8P\x01V\xec\xbfE\xc7o7\xbcX9\xbd[\x97\xbd\xaa\x7f\xf2\x9f\\D\xfeX\xc9`\x96\xf6\xd9H\xc6\xb1\xff\xb3}\xb1g\xe3\xa2\x15J&\x17K_EP\xf3g^I\xf4\x9fD\xc6\xd6\x9b\xe1\xbf\x870\xd46\x8c\x96\xa4\x9c\xb1#\xdd\xe6\xee\xb2;4A\x82i\x806(\x05\xee)\xad\x85\x00\xbe\xbb\xb0\xbfJv\xe0\xdeSNM^m_\xfb|\xe5\xf4\xbcA\x0fu\xa3*4\x91\xbc\xe6\xabP*dqb\xe8\x7f\xb3\xd5\xba\x08\x8c\xb9\x8c\x86:n\xf8|0\x97CD\xb1A\x91\xdd\xc1\xd6\xa7x\xb1\x06\x0f\xe7\xb3\x82P@\x14\xba\xb8\x11\x1b\xb6\x871\xf5\xdb\x1b\xe2\x8b\x06D\x8a\xd4c;\xb8\xf5\xea\xddg\xf3\xf5W\x18\xdb\x989\xadp\xb5\x83\xed-\re\x15\x9d\\\xb9\x91\x8d\xab\x84\xdd\xed\x9f\xeb]\xcd\xfd_\'M\x9b\x1f\xba\xd7\xd8$\x8a\xb6\xd7qb\xd4\xf2\xcc\xf3\xa5\xd1\xee\xf3!\xd9\x92\x81\x10\xc6\x86\xf6\x1cs\xc5\xb2<\xd7t\x92,\xc8\xa6\x19\xe1^\xd8\xc45(\xe2\xc4\x94\xf7\xb8w\xea\xdf\x00r\xb3\xfc\xee\xd0\xfdB\xc2\xe4Q\x17\xcbgA\xdc\x84$5\xe6#\xbc\x02/\xc3p\x9cY>jb\xf6\xc8 \xa6\xe2\x87_\x07\xc0mW].\xc6\x8c\x01g\x8a\xf1\x15l\xe0j\xe3\x19\xb5\x99\xb6\xef\xeb\xd4F\x8c\xdd?\xae\xd4UK\xf4\xcd\r\xb9\xb0e\x18\xe9\x94T~\xf0\xd8\xf7\xd0m\xc5^Z\x8e\x168\x06N\xaf\xf8\\\xe0\xd3\xcd\x01\x04|\x11\x96\xe6xM\x1c\xc8\x18#s\xa9\xac\xe5\xcf;\x044\x96\xc2D\xfc\xef*\x1fc\xc9\x07U\x93\x0cfPP\x06\xa8\x0b4\x91H\xe2\xc4\xa0z\xb5{\xc9\xe8\x0e\x16\xd2 _\x0b\xcd\xa2\x86b4`v\xc2\xaf\xb5\x03z\xbacK\xa5\xf5a\r\t\xdaU\xa4me\n\x89"nV\xcaK\xb7\xec|b)\x0er\xdd\xd0\xfc\xf1\xb3\x1b\x82^J\x06,\xa5\xbf\xa1\x18\xa8\x84\xa4\xdd\xc1\xd8\xdc\xcfz8l1u9\xf8_sw\x14gk\xfe\xb2\xf3H\r\x95\xf0c\xa5\xf4\x8f\x81\t\x02F\xb0\x7fK\xd5\x9aV8\x92\x83\x96P\x9f\x9f\x1f\xdd\x91\x9f\x82\xea\x1a\x80E\xd1\xc8\x81\x9eM{2\xe6G/\x1d@a\xb9I\xdb\x003\xd4\xc1\xb5\xa9\xa5\xefK\x81\xa2\xa5d\x11\xaf\x96X\x82\xb2\x85]\x0c\xa1\xc9\xf0\x10\xc6\xe2v\x85\xef\x8c\x1d@Lo\xfa5\xf9\xfa\xb2t(\x01\x03z6\xdf\x9d#c\x80Z3\xc5I\xb3}\xbb\t\x94\x13oo\xfc\x81\x8dD\xdf&p\xc6_3\x9494\xd3s\xed\xc7\xb5\x06\xc5\xa1v\xc3\xd0\xda=\xc6Nm\xb0\xa1\xb1k[d9\xa3\xdc\x8a\xe9d\xed\x02\x9f\x0er\x0529r<\x9db\xb0\x8b\x17\x17U\x81\xe2\x8b4\xb7\xe5\x07\xce\\\xf5\x05\xb5)\x8f\xa2:\xf3\xe2?\x9d\xe7\xe6\x9e#\xb0/B\xe4\xac\xe6\x8f\xe3\x1a7\x94#\xb3\xa1\xd5 \xdcR9\xa5sC\xbesM\x1a\x01\x8a\xc4\x83\xabo\xe3\xd4\xfd5\xd4\t\x91\x90\x07\n\xf1k\x97\x12\x91\xaf\xa1\x12\x14\x0b\x95\xb5\\AV\xaf\xbaQ\xe0\xdeF\x08$\xed\x7fa\x14\x11@\xcb<%\xa3\x08\x01;\x00\xda\xc6\xa7\xc3H\x1f\xe5C\xa3F:\x0cw\xda\x9a\xf8\x8a?\x1fmT\x12\x8a\xd2\xa4\xfd\xe6\xa8\xb1\x1a\xfe\xf9G\xeb,\xbbF\x04\x99\x18\xd3:\r\xe2\xd4J\x08\x1b\x9a\x16Sk!\xc4\xa80\xfb\x18\xb2s\xd6\xa2\xe0&\x87\x81\x1d\xa3\xe3:<\x93w\xc37\x19\x07\x8dJ\x1ca\x04D\xe7!\x14:\x08"\xb7"\xc2\x10`\x15\xd0\xdeQ\xc9\xd4\xce[=\xb7\x14(G{,\xa5z]\xe2p\x11W a\xf4\xcd\xe1_\xa0\xa6\xde\x81WhO\xca\x1dfn\x05\x8f}\xf1h[B\xd4z\xdeR\x1a\xc2\x8f\xbd\xe28X\x18%\xfc\xb8\xf6\xa9B5n\x10\xd6zr\x90s\xfe\x07\x05HH\x9e\xfc\x8d\x92\x94\x9bx\xb9\xaa\xd9\x81\tL\x88\xd0\xd9\xbca}\x10\xbf\xe8\x07%\xf2\xc2\xdc]0@\x18J\x18\x11\xac\x16\xd90>B\xc5\xb1\n\xa4\x9ae\x19>>\xc8_\xaa\xb7P\xe3\xe2\xa5^\xdd\xea\x1d\xe3n\xc2\xf7\x17\n7=@\x9b\xd5\x01X\xf4\x82y%\x03\xbf\xc9N\xe9\xc5\x81YW\xf8\xe6\xc2\x17~A\x92]\x9b\xba-\xbaV\x18\xda\x05\x98\x86\x1c\xf4\xe1\xba\xb1\re\x90e5\x85\x9a\xb0^\x82\x80\xab\x1b\x13\xd3\xcf\xd91\xbbJ\x0f\xbcP\x19\x83\xa8\'-\xe5M\x8aA\xf8\xdb\x96{\xa7g\x0b\xa9\xa1\xfd\xc8\xaf\x94\xb1}U\x00\xd6\x1e\x8c\xeb1g\xe4\xbf\x9d\xd6\xe4\xdf\xfc\xe4\xc4\xdc\xa3\xe3\x15\'\xc26\xde\x87\xbc f\xf7\x96\xf1HCNB\x9c\\\xf8\xc0\xfe\xfd\xb3Z\xd5NoI\x05\xfd\xabT\x1e\x1c\xe1\xd0\x87tF\x1b/^\x94\xdb\x0c\xbe\xd7:\x7fdr\\\x98\xc8q\x83:\xee\x83\xcb\xef\x9e\x03\xf9\xc3\xdb\x17\xd7w\xaf\xa2ax\x8e\t\xb3\x02-\xdfq\xf8X\xdc]\x9cL:\x8d\xf8E\x1bT\xc6fV\x13\xc9]\xf9J\xec\x18cD_\xe4U\x7f\xcd[<"\xa6a\xe5\x0b|\x14\xa1\xb1j\x8c\xd1\xc0\x90\x10D\xc1\xe4\xa9\x88\x9f\xf6<\xb6g5\xf5\xe3n\x08\xbf\xa9X\xfcM\xc0\xa3\x02\xb3d\xc9*\x1a\x0e\xa7\x96J\xf3\xf7\xd3m\xbb\x1f\xdbmd\xbc\x04\x13\xf5\xd7\xec\x85\xcf\x08k\xc5\xf0z\x16\xaf\x8b~\x19\xb2$\x03\xbas\x1e\xaeY\xc7\xd9^\xdf\xd2\xfc\xcb\xae\x7f>\xc5\x93;\x84\xe7\xd8\x9b\xd3x\x1a\xda\xd97\x93\x91g!3\x12\x00\xe7\x81g\x87\x16\xd3\x8b\xf8!\xcbZB\xb5:o\x15\xf7\xcc>\xbc\xca\x89B:\x99\x8d"\xe2\xbfkg\x81\t\x98_\x1c\xc9\xe9\x08P[\x9b\xb1\xf7:\xca-\xe3S\x1a3\x98 \x94\xccz\x85\x96\xf4?\x9a\xfa\x88\xb7.\x0f\x1d\xfa\\\x86#\xdf\x8a\xd0\xe3+{\xf9\xe0\t\x88$%>p\xcc5H\xb5 "6_\x96\x92\xf4\x15\xc5\x9f|\x18\xf2\xa9C\xd6\xcf\x86\x86F\x11\xfe4\xa1B\xd1\xc8\x888\xfek\xbf\x90Vb&}\x8f4Ne\xe4y\xe9\xec\xac/\x06$\xa507_\xd9P\xe1\xef\xb9Z;\x86"\x17\xdd*\x00\xb4]\xfc*\xa8\xdd\xfb\x07\xe5gr\x0cmE.\xa4\xd8\x88\xc4\xb8\xd9c=\x98\x10\x0eu\xf0\xb4\x90\xdc\xf9\x86\xbc\xb7\xf7\xae()\x13R\xc1\xa9\xbet\xd9\xecksL\xfd\xd6\xca\xea\xdb\xf26A\xf5\x11\xd8\x82o\xefFV/T-\xad\xde\xa7\xbe\x7f\xe3@\x19UH\xef\x9a.%6\xa6*\xa6\xc8\xf4\xb5\xcb\x18J\xd8\xdd\xce\x12\xcd\xcfK\x92\xcb\x19\x1fTg\xeed\x8b)(\'h\x82w\x0e\xb5\xedugb\x1f\xa4\xac\xb0\x11\x9aH\xec\xf8_>X\x980\xd28\xee\xbd\x7f\x81H_v\x12\x8f\x92U\x17\xbd\xbeq\x073\x8a\xc3Jem8\xa2\xc3c\x99\xceB\x1cH\xe4f\xa0\xc4\x07\xd2"GF]-\x1d\x86\xd5v\xc9\xc1\x05\xbc\xdf\xea\x1fU\xa5\xe0\x85\xe7{\x8f\x0f\xf3\xdd\xce\x8c\xd0<%\x9f\xfex\x06\x8fV\x03\xc9\x02d\x7f\x93\xdd\xe0I\x83\xce\xd4\x1b\xd34SP\x06\x83\xff\xb3\xfc\x19\x01\xd1\xb8\xe3\xa9\x1f\xd0\x1f$\xaf+\xc7\xf9\x13\x85`\x94\x91f\xc9\xd4\xdbf\xa0\xe7\xc2R\xb0I\xfb\xeb\x90]6vx\x06\xdd5-a\x96\xee\xce\xb6\x87\x9d!\xc5\x02h0xQ\xdcT\x14\xa7k\xd6;V\x02\xbcY3\x1a\xd0\xbbCs\\\x1a\xa87\xd1\xbc\xd3\xf4ks\xc4\xe2\xe6l\xa9R\x8eX\x81\xee{\x0e\x17\xb1\x91\xf1\x93\x1e]\xc3\xf1/,t(\xfe\xc8_\xeaZT\xc0o\x99\x8b\xdfU\x87}I\xa5\xa6<\x17h\xe6\x89k\xcf\xc1\x00\x93F( \x9c\x07\xeb|\xe29\x94j\x96\x99\x87\xebgw\xdd\x99\x80\xf3]\xfdFgsT>\xeaH\xda\xdd\xc2\xb3S\xac\xb32\x8d\xdc\\V\x83\x10V\xf2\xcb\xaaQ\xa0\xf0v\xb3c<\x9cN\x1f"\xfbF\xc2*4E/l\x89\xf6\xf8\xa4\x97u7^n\xe2\x1fj\xf3#\x8f\x1a\x86\xbd\xef\x01f\xd6/~p\xabz\x0e\xd7\x8f\xf1#\x8b\x18\xad\xd9fnk0\xa5\x18j\x94?\xd0\x91\x8csr\x8b\xdb\x19\xd8\xcb4\xa8\xe0\x11\xeaN\xbbJW\xce\xca\x1b\x01\xab}\x87\x82\xc2\x01:!\x1e\xb2\x99P\xdc\xe0\xadV\x03\xfe:){\xaa\x10\x19\xf4\x84+\x11\xa6=*\x06\xf0\xa4wP!\xb9S\x96\xf2\x0b\xacM\xff\x06tZZPc\xe9O,\xab-\xfc\x1bbx\x8b=v{ \xdb\xf8\x12\xcb\xe9\xb10\xf3C\x0ff\xea\xf0n\xaf>qH\xb7\xa6?\xe3\x1c\x12\x1f\x15\xd0XiA\xb4Q\xcf\xd0\x94q\x00\x0c\xa7\xbc4\xb5s$\x13\xe1\xa6\xc4A\xf7\x83H\x88\xcf\xa0\xa8Y\x90\xa1\xfa"p\xb6\xae\xd0b\xc7\x92"\xd7\x08\x9d8`\xeb\x82\x19:\x7f\xba\xb4-/\x1d\x84L^\xc0\xafH>\xca>\xb8\xc9\x06x\xd3\xe6\xb8\x00\n=\xf4D\x05\xd9j\xaaE)\xf4\x06%\x90\xe0\x1fn\xf72\xb1o\x81\x91n\xfa\x1f|\x08\xb6EN\xab\xd1B\xb4L\x1am%\x12O;\xfd\xd0\'j~\xb6\xbc\xca\xc9\xb2\x8ao\xc2\xf4\x84\\\x9c\xe0H\n\xa8l\xd7i)&\xce%_\x9b@\xe3\xee_\x1d\x0f\x906\xaa-\x11\x92\xdc\xb6\xff\xaf\x81e\xc3\x0e\xc6\x8c:\xc6\x15|XXNv\xff\x83\x9aY\x1f\xf8\x87\x810\xb9\x91\xd6+\x04\x89{\x8c\x92\x85\xa3\xf5\xcf8\xef\x8f\xf1x\'IJ u/\x1e\xce\xb9@\x82\x0e\x1c\x1c\x17\xad`\x13z5M\xde[\xf8\x98n\xa8\x94\xe4k\xb1@\xd9\rF\xbem\n\xd6I\xce5\xd0\xce\xb7-\xf2\xbbb\xbf\t9\xf3\x16\xc5\x10E\x97|\x08\xf7zl\xb5%\xbcv\x95\xd95\x95\xd80\x02F\x03\xe1G\xc0\xe3\xfd\xf5\\\xd4\xa7\xe0\\8\xfa)J\xd2n\x95\x9b\xbf\xd4K\xb9\xf9\x93\xf3\xd6\xf3p1\xee\x8c(\x9b\x19\x01\xd0\xf8\xf0\x0eZ\x0b\xab\x80/\xafHZK\x0e\xca\xb2\xe6U\x99`\xef-\xf1)\x96\x1e\x99\x99\x03L\x11\xc9:\xa5"\xe2s\xb7$\x89\x9d8\tJ\xdf\x1dm]\xb6\x07\xa6T\r"\x01\x19\x8f3\xcb)T)\xf0\xbeI\xf7\xde\xfe\xef\x1c\\\x8e\xf0\x9d\xf25Y\x0bU\xcf\x9e[\x94I\xf8\xf0\x94\xe1\xb1M\xcc\x91\x93\xee\x1d\xcfY'
encrypted_request = post_data[128:]

round1 = rc4(rand_rc4_key, encrypted_request)
binstorage = visual_decrypt(round1)

binstorage_header = parse_binstorage_header(binstorage)
print("request binstorage_header:\n")
pprint.pprint(binstorage_header)

binstorage_sections = parse_binstorage_sections(binstorage[48:], binstorage_header["count"])
print()
print("request binstorage_sections:\n")
pprint.pprint(binstorage_sections)

fp = open("enc_dynamic_config_response.bin", "rb")
response = fp.read()
fp.close()
round1 = rc4(rand_rc4_key, response)
binstorage = visual_decrypt(round1)

binstorage_header = parse_binstorage_header(binstorage)
print("\nresponse binstorage_header:\n")
pprint.pprint(binstorage_header)

binstorage_sections = parse_binstorage_sections(binstorage[48:], binstorage_header["count"])
print()
print("response binstorage_sections:\n")
pprint.pprint(binstorage_sections)

request binstorage_header:

{'count': 18,
 'flags': 0,
 'md5Hash': b'\xe3\x12\xb4\x9a@\x08\xe4e\xbe\x07\xb8\x9cE\xd7\xf3\xba',
 'randData': b'\xa4a\xcb\x9d[\x80\x90\xbc0\xcfv\xce\xc6\xa5W\x08\xd9\xbc(\xa6',
 'size': 3072}

request binstorage_sections:

[{'data': b'Zep', 'flags': 0, 'id': 10002, 'realSize': 3, 'size': 3},
 {'data': b'M2', 'flags': 0, 'id': 10025, 'realSize': 2, 'size': 2},
 {'data': b'WIN-PDDC81NCU8C_Administrator_63EFF4E8',
  'flags': 0,
  'id': 10001,
  'realSize': 38,
  'size': 38},
 {'data': b'\x00\x04\x06\x02',
  'flags': 0,
  'id': 10003,
  'realSize': 4,
  'size': 4},
 {'data': b'\x0c\x00\x80%\t\x00',
  'flags': 0,
  'id': 10012,
  'realSize': 6,
  'size': 6},
 {'data': b'\x02\x00\x00\x00',
  'flags': 0,
  'id': 10023,
  'realSize': 4,
  'size': 4},
 {'data': b'\x02\x00\x00\x00',
  'flags': 0,
  'id': 10024,
  'realSize': 4,
  'size': 4},
 {'data': b'\n\x00\xe8F', 'flags': 0, 'id': 10016, 'realSize': 4, 'size': 4},
 {'data': b'\x03\x00\x00\x00',
  'flags': 0,
  '